In [1]:
!kubectl get nodes --show-labels

NAME      STATUS   ROLES    AGE     VERSION   LABELS
master    Ready    master   5h34m   v1.18.5   baz=None,beta.kubernetes.io/arch=amd64,beta.kubernetes.io/os=linux,foo=bar,kubernetes.io/arch=amd64,kubernetes.io/hostname=master,kubernetes.io/os=linux,node-role.kubernetes.io/master=
worker1   Ready    <none>   5h34m   v1.18.5   beta.kubernetes.io/arch=amd64,beta.kubernetes.io/os=linux,kubernetes.io/arch=amd64,kubernetes.io/hostname=worker1,kubernetes.io/os=linux


In [44]:
from pprint import pprint
from kubernetes import client, config

def set_node_labels(node_name='minikube'):
    config.load_kube_config()

    api_instance = client.CoreV1Api()

    body = {
        "metadata": {
            "labels": {
                "label1": "value1", # OK
                "label2": "value2", # OK
                "emptylabel": None  # Not working
            }
        }
    }

    api_response = api_instance.patch_node(node_name, body)

    pprint(api_response)
    
set_node_labels('master')

{'api_version': 'v1',
 'kind': 'Node',
 'metadata': {'annotations': {'kubeadm.alpha.kubernetes.io/cri-socket': '/var/run/dockershim.sock',
                              'node.alpha.kubernetes.io/ttl': '0',
                              'projectcalico.org/IPv4Address': '172.31.14.166/20',
                              'projectcalico.org/IPv4IPIPTunnelAddr': '192.168.219.64',
                              'volumes.kubernetes.io/controller-managed-attach-detach': 'true'},
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2020, 7, 13, 7, 22, 34, tzinfo=tzlocal()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'initializers': None,
              'labels': {'baz': 'None',
                         'beta.kubernetes.io/arch': 'amd64',
                         'beta.kubernetes.io/os': 'linux

In [7]:
!kubectl get nodes --show-labels

NAME      STATUS   ROLES    AGE     VERSION   LABELS
master    Ready    master   5h36m   v1.18.5   baz=None,beta.kubernetes.io/arch=amd64,beta.kubernetes.io/os=linux,foo=bar,kubernetes.io/arch=amd64,kubernetes.io/hostname=master,kubernetes.io/os=linux,label1=value1,label2=value2,node-role.kubernetes.io/master=
worker1   Ready    <none>   5h36m   v1.18.5   beta.kubernetes.io/arch=amd64,beta.kubernetes.io/os=linux,kubernetes.io/arch=amd64,kubernetes.io/hostname=worker1,kubernetes.io/os=linux


In [81]:
def main():
    #config.load_incluster_config()
    config.load_kube_config()

    corev1 = client.CoreV1Api()
    print("Listing pods with their IPs:")
    ret = corev1.list_pod_for_all_namespaces(watch=False)
    for i in ret.items:
        print(f"{i.metadata.namespace:12s} {i.metadata.name:42s} {i.status.pod_ip:16s} {i.status.host_ip:16s}")
        #print("%-12s  %-42s %16s %16s" % (i.metadata.namespace, i.metadata.name, i.status.pod_ip, i.status.host_ip))

main()

Listing pods with their IPs:
default      ckad-demo-76bb8d757-2594q                  192.168.219.105  172.31.14.166   
default      ckad-demo-76bb8d757-6clm9                  192.168.219.103  172.31.14.166   
default      ckad-demo-76bb8d757-7mnpr                  192.168.235.150  172.31.15.1     
default      ckad-demo-76bb8d757-98fj7                  192.168.219.104  172.31.14.166   
default      ckad-demo-76bb8d757-dt4wg                  192.168.235.146  172.31.15.1     
default      ckad-demo-76bb8d757-fswh7                  192.168.235.147  172.31.15.1     
default      ckad-demo-76bb8d757-krjdl                  192.168.219.102  172.31.14.166   
default      ckad-demo-76bb8d757-pngxk                  192.168.235.143  172.31.15.1     
default      ckad-demo-76bb8d757-q75ks                  192.168.235.144  172.31.15.1     
default      ckad-demo-76bb8d757-rvldm                  192.168.235.145  172.31.15.1     
default      ckad-demo2-7f9b6f49d5-9k2pt                192.168.235.185

In [82]:
config.load_kube_config()
#config.load_incluster_config()
#v1 = None
corev1 = client.CoreV1Api()
appsv1 = client.AppsV1Api()
batchv1 = client.BatchV1Api()
batchv1beta1 = client.BatchV1beta1Api()

In [83]:
dir(client)

['AdmissionregistrationApi',
 'AdmissionregistrationV1beta1Api',
 'AdmissionregistrationV1beta1ServiceReference',
 'AdmissionregistrationV1beta1WebhookClientConfig',
 'ApiClient',
 'ApiextensionsApi',
 'ApiextensionsV1beta1Api',
 'ApiextensionsV1beta1ServiceReference',
 'ApiextensionsV1beta1WebhookClientConfig',
 'ApiregistrationApi',
 'ApiregistrationV1Api',
 'ApiregistrationV1beta1Api',
 'ApiregistrationV1beta1ServiceReference',
 'ApisApi',
 'AppsApi',
 'AppsV1Api',
 'AppsV1beta1Api',
 'AppsV1beta1Deployment',
 'AppsV1beta1DeploymentCondition',
 'AppsV1beta1DeploymentList',
 'AppsV1beta1DeploymentRollback',
 'AppsV1beta1DeploymentSpec',
 'AppsV1beta1DeploymentStatus',
 'AppsV1beta1DeploymentStrategy',
 'AppsV1beta1RollbackConfig',
 'AppsV1beta1RollingUpdateDeployment',
 'AppsV1beta1Scale',
 'AppsV1beta1ScaleSpec',
 'AppsV1beta1ScaleStatus',
 'AppsV1beta2Api',
 'AuditregistrationApi',
 'AuditregistrationV1alpha1Api',
 'AuthenticationApi',
 'AuthenticationV1Api',
 'AuthenticationV1beta

In [84]:
def get_nodes():
    nodes = {}

    ret = corev1.list_node(watch=False)
    for i in ret.items:
        node_ip   = i.status.addresses[0].address
        node_name = i.metadata.name
        #print("%-12s  %-16s" % (node_name, node_ip))
        print(f"{i.metadata.name:12s} {node_ip:16s}")
        nodes[node_ip] = node_name
    return nodes
        
#print("Listing nodes with their IPs:")
nodes = get_nodes()

master       172.31.14.166   
worker1      172.31.15.1     


In [85]:
def get_pods(namespace='all'):
    if namespace == 'all':
        ret = corev1.list_pod_for_all_namespaces(watch=False)
    else:
        ret = corev1.list_namespaced_pod(watch=False, namespace=namespace)
    for i in ret.items:
        pod_name = i.metadata.name
        pod_namespace = i.metadata.namespace
        pod_ip = i.status.pod_ip
        host_ip = i.status.host_ip
        host = host_ip
        if host_ip in nodes:
            host = nodes[host_ip]
        print(f"{i.metadata.namespace:12s} {i.metadata.name:42s} {i.status.pod_ip:16s} {i.status.host_ip:16s}")
        #print("%-12s  %-42s %16s %16s" % (pod_namespace, pod_name, pod_ip, host))

print("[all namespaces] Listing pods with their IPs:")
get_pods()

print()
print("[namespace='default'] Listing pods with their IPs:")
get_pods(namespace='default')

[all namespaces] Listing pods with their IPs:
default      ckad-demo-76bb8d757-2594q                  192.168.219.105  172.31.14.166   
default      ckad-demo-76bb8d757-6clm9                  192.168.219.103  172.31.14.166   
default      ckad-demo-76bb8d757-7mnpr                  192.168.235.150  172.31.15.1     
default      ckad-demo-76bb8d757-98fj7                  192.168.219.104  172.31.14.166   
default      ckad-demo-76bb8d757-dt4wg                  192.168.235.146  172.31.15.1     
default      ckad-demo-76bb8d757-fswh7                  192.168.235.147  172.31.15.1     
default      ckad-demo-76bb8d757-krjdl                  192.168.219.102  172.31.14.166   
default      ckad-demo-76bb8d757-pngxk                  192.168.235.143  172.31.15.1     
default      ckad-demo-76bb8d757-q75ks                  192.168.235.144  172.31.15.1     
default      ckad-demo-76bb8d757-rvldm                  192.168.235.145  172.31.15.1     
default      ckad-demo2-7f9b6f49d5-9k2pt              

In [86]:
def get_deployments(namespace='all'):
    if namespace == 'all':
        ret = appsv1.list_deployment_for_all_namespaces(watch=False)
    else:
        ret = appsv1.list_namespaced_deployment(watch=False, namespace=namespace)
    for i in ret.items:
        #deploy_name = i.metadata.name
        #deploy_namespace = i.metadata.namespace
        #print("%-12s  %-42s" % (deploy_namespace, deploy_name))
        print(f"{i.metadata.namespace:12s} {i.metadata.name:42s}")
        
print("[all namespaces] Listing deployments:")
get_deployments()

print()
print("[namespace='default'] Listing deployments:")
get_deployments(namespace='default')

[all namespaces] Listing deployments:
default      ckad-demo                                 
default      ckad-demo2                                
kube-system  calico-kube-controllers                   
kube-system  coredns                                   

[namespace='default'] Listing deployments:
default      ckad-demo                                 
default      ckad-demo2                                


In [87]:
def get_daemon_sets(namespace='all'):
    if namespace == 'all':
        ret = appsv1.list_daemon_set_for_all_namespaces(watch=False)
    else:
        ret = appsv1.list_namespaced_daemon_set(watch=False, namespace=namespace)
    for i in ret.items:
        #deploy_name = i.metadata.name
        #deploy_namespace = i.metadata.namespace
        #print("%-12s  %-42s" % (deploy_namespace, deploy_name))
        print(f"{i.metadata.namespace:12s} {i.metadata.name:42s}")
        
print("[all namespaces] Listing daemonsets:")
get_daemon_sets()

print()
print("[namespace='default'] Listing daemonsets:")
get_daemon_sets(namespace='default')

[all namespaces] Listing daemonsets:
kube-system  calico-node                               
kube-system  kube-proxy                                

[namespace='default'] Listing daemonsets:


In [88]:
def get_replica_sets(namespace='all'):
    if namespace == 'all':
        ret = appsv1.list_replica_set_for_all_namespaces(watch=False)
    else:
        ret = appsv1.list_namespaced_replica_set(watch=False, namespace=namespace)
    for i in ret.items:
        #deploy_name = i.metadata.name
        #deploy_namespace = i.metadata.namespace
        #print("%-12s  %-42s" % (deploy_namespace, deploy_name))
        print(f"{i.metadata.namespace:12s} {i.metadata.name:42s}")
        
print("[all namespaces] Listing replicasets:")
get_replica_sets()

print()
print("[namespace='default'] Listing replicasets:")
get_replica_sets(namespace='default')

[all namespaces] Listing replicasets:
default      ckad-demo-76bb8d757                       
default      ckad-demo2-7f9b6f49d5                     
kube-system  calico-kube-controllers-578894d4cd        
kube-system  coredns-66bff467f8                        

[namespace='default'] Listing replicasets:
default      ckad-demo-76bb8d757                       
default      ckad-demo2-7f9b6f49d5                     


In [90]:
def get_services(namespace='all'):
    if namespace == 'all':
        ret = corev1.list_service_for_all_namespaces(watch=False)
    else:
        ret = corev1.list_namespaced_service(watch=False, namespace=namespace)
    for i in ret.items:
        #deploy_name = i.metadata.name
        #deploy_namespace = i.metadata.namespace
        #print("%-12s  %-42s" % (deploy_namespace, deploy_name))
        print(f"{i.metadata.namespace:12s} {i.metadata.name:42s}")
        
print("[all namespaces] Listing services:")
get_services()

print()
print("[namespace='default'] Listing services:")
get_services(namespace='default')

[all namespaces] Listing services:
default      ckad-demo                                 
default      kubernetes                                
kube-system  kube-dns                                  

[namespace='default'] Listing services:
default      ckad-demo                                 
default      kubernetes                                


In [91]:
def get_jobs(namespace='all'):
    if namespace == 'all':
        ret = batchv1.list_job_for_all_namespaces(watch=False)
    else:
        ret = batchv1.list_namespaced_job(watch=False, namespace=namespace)
    for i in ret.items:
        #deploy_name = i.metadata.name
        #deploy_namespace = i.metadata.namespace
        #print("%-12s  %-42s" % (deploy_namespace, deploy_name))
        print(f"{i.metadata.namespace:12s} {i.metadata.name:42s}")
        
print("[all namespaces] Listing jobs:")
get_jobs()

print()
print("[namespace='default'] Listing jobs:")
get_jobs(namespace='default')

[all namespaces] Listing jobs:

[namespace='default'] Listing jobs:


In [92]:
def get_cronjobs(namespace='all'):
    if namespace == 'all':
        ret = batchv1beta1.list_cron_job_for_all_namespaces(watch=False)
    else:
        ret = batchv1beta1.list_namespaced_cron_job(watch=False, namespace=namespace)
    for i in ret.items:
        #deploy_name = i.metadata.name
        #deploy_namespace = i.metadata.namespace
        #print("%-12s  %-42s" % (deploy_namespace, deploy_name))
        print(f"{i.metadata.namespace:12s} {i.metadata.name:42s}")
        
print("[all namespaces] Listing cronjobs:")
get_cronjobs()

print()
print("[namespace='default'] Listing cronjobs:")
get_cronjobs(namespace='default')

[all namespaces] Listing cronjobs:

[namespace='default'] Listing cronjobs:


In [43]:
methods = [ method for method in dir(v1) if "reso" in method]
print("\n".join( methods ))

create_namespaced_resource_quota
create_namespaced_resource_quota_with_http_info
delete_collection_namespaced_resource_quota
delete_collection_namespaced_resource_quota_with_http_info
delete_namespaced_resource_quota
delete_namespaced_resource_quota_with_http_info
get_api_resources
get_api_resources_with_http_info
list_namespaced_resource_quota
list_namespaced_resource_quota_with_http_info
list_resource_quota_for_all_namespaces
list_resource_quota_for_all_namespaces_with_http_info
patch_namespaced_resource_quota
patch_namespaced_resource_quota_status
patch_namespaced_resource_quota_status_with_http_info
patch_namespaced_resource_quota_with_http_info
read_namespaced_resource_quota
read_namespaced_resource_quota_status
read_namespaced_resource_quota_status_with_http_info
read_namespaced_resource_quota_with_http_info
replace_namespaced_resource_quota
replace_namespaced_resource_quota_status
replace_namespaced_resource_quota_status_with_http_info
replace_namespaced_resource_quota_with_http